In [16]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from torch_geometric.data import Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from datetime import datetime
import networkx as nx
from sklearn.metrics import precision_score, recall_score, f1_score
from torch_geometric.loader import LinkNeighborLoader

from util import get_node_encoder
import os
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import json
import node2vec
from collections import Counter, defaultdict
from gensim.models import Word2Vec

In [17]:
class LinkClassifier(torch.nn.Module):
    def __init__(self, hidden_channels=128, embedding_dim=64):
        super(LinkClassifier, self).__init__()
        
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(embedding_dim * 2, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels, hidden_channels * 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels * 2, 2),
        )
        
    def forward(self, u, v):
        h = torch.cat([u, v], dim=1)
        return self.mlp(h).squeeze(1)
        
class LinkRegressor(torch.nn.Module):
    def __init__(self, hidden_channels=128, embedding_dim=64):
        super(LinkRegressor, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(embedding_dim * 2, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels, hidden_channels * 2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels * 2, 1),
        )
        
    def forward(self, u , v):
        h = torch.cat([u, v], dim=1)
        return self.mlp(h).squeeze(1)

class DownStreamModel(torch.nn.Module):
  def __init__(self, hidden_feats = 128, embedding_dim = 64):
    super().__init__()
    self.cls_head = LinkClassifier(hidden_feats, embedding_dim)
    self.reg_head = LinkRegressor(hidden_feats, embedding_dim)
    
  def forward(self, embeddings, id_1, id_2):
      
    dst_z = embeddings[id_1]
    src_z = embeddings[id_2]
    
    logits = self.cls_head(dst_z, src_z)
    scores = self.reg_head(src_z, dst_z)
    
    return logits, scores

In [18]:
# 读取 k_pop_group_info 数据
with open("/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/artist_texts.json", "r") as f:
    node_info = json.load(f)
    
max_length =0
for node in node_info.keys():
    if len(node_info[node]) > max_length:
        max_length = len(node_info[node])
max_length

34717

In [19]:
model = DownStreamModel(embedding_dim=512).to('cuda')

# 加載數據
collaboration_df = pd.read_csv("/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/data/collaboration_videos.csv")
collaboration_df['timestamp'] = collaboration_df['timestamp'].astype(float)

In [20]:

WINDOW_SIZE = 180
REMOVED_SAME_GROUP = False
Q = 1
P = 0.5

node_encoder = get_node_encoder()


In [21]:
num_epochs = 200
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


# 設定 early stopping 參數
patience = 10
min_delta = 0.001
best_loss = float('inf')
patience_counter = 0



# 只使用不同組的藝人
if REMOVED_SAME_GROUP:
    collaboration_df = collaboration_df[collaboration_df['source'].str.split('_').str[0] != collaboration_df['target'].str.split('_').str[0]]
    
for epoch in range(num_epochs):
    epoch_loss = 0
    end_time = collaboration_df['timestamp'].min()
    offset = 10
    total_iter = 0
    while end_time < collaboration_df['timestamp'].max():
        start_time = end_time - WINDOW_SIZE * 24 * 3600
        window_data = collaboration_df[
            (collaboration_df['timestamp'] >= start_time) & 
            (collaboration_df['timestamp'] < end_time)
        ]
        
        
        offset = len(collaboration_df[collaboration_df['timestamp'] <= end_time]) + 1
        
        if offset >= collaboration_df.shape[0]:
            break
        
        end_time = collaboration_df['timestamp'].iloc[offset]
        
        if len(window_data) < 10:
            continue
    
    
        # 創建邊索引和特徵
        edge_index = []
        scores = []
        
        for _, row in window_data.iterrows():
            artist1_idx = node_encoder.transform([row['source']])[0]
            artist2_idx = node_encoder.transform([row['target']])[0]
                
            edge_index.append([artist1_idx, artist2_idx])
                
            # 計算邊特徵
            views = row['views'] if 'views' in row else 0
            likes = row['likes'] if 'likes' in row else 0
            comments = row['comments'] if 'comments' in row else 0

            scores.append(row['views'] * 0.5 + row['likes'] * 0.3 + row['comments'] * 0.2)

        G = nx.DiGraph()
        
        # 建立節點
        for i in range(len(node_encoder.classes_)):
            G.add_node(i)
        
        # 计算边的权重
        idol_values = defaultdict(list)
        for _,row in window_data.iterrows():
            artist1_idx = node_encoder.transform([row['source']])[0]
            artist2_idx = node_encoder.transform([row['target']])[0]
            effectiveness = row['views'] * 0.5 + row['likes'] * 0.3 + row['comments'] * 0.2
            
            combination = (artist1_idx, artist2_idx)
            
            idol_values[combination].append(effectiveness)
        
        # 计算平均效益
        idol_avg = {name: sum(vals) / len(vals) for name, vals in idol_values.items()}
    
        # 更新图的权重
        for name, effectiveness in idol_avg.items():
            source_name, target_name = name
        
            distance = float('inf')
            if effectiveness and effectiveness > 0:
                distance = 1 / effectiveness
        
            G.add_edge(source_name, target_name, weight=distance, effectiveness=effectiveness)
        
        vec_G = node2vec.Graph(G, is_directed=True, p=P, q=Q)
        vec_G.preprocess_transition_probs()
        walks = vec_G.simulate_walks(num_walks=3, walk_length=80)
        walks = [list(map(str, walk)) for walk in walks]
        embedding_model = Word2Vec(walks, vector_size=512, window=20, min_count=0, sg=1, workers=8, epochs=5,seed=42)
        
        # 轉換為張量
        node_features = torch.tensor(embedding_model.wv.vectors, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        scores = torch.tensor(scores, dtype=torch.float).t()
        
        # 創建圖數據
        data = Data(x=node_features, edge_index=edge_index, scores=scores).to('cuda')
        
        # 使用 LinkNeighborLoader 進行正負取樣
        loader = LinkNeighborLoader(
            data=data,
            num_neighbors=[10, 10],  #每個節點採樣10個一階鄰居和10個二階鄰居
            batch_size=32,
            edge_label_index=edge_index,  # 正樣本邊
            neg_sampling_ratio=1.0,  # 負樣本比例為1:1
            shuffle=True,
        )

        # 訓練模型
        model.train()
        optimizer.zero_grad()
        for batch in loader:
            batch = batch.to('cuda')
            optimizer.zero_grad()
            
            source_idx = batch.edge_label_index[0].t()
            target_idx = batch.edge_label_index[1].t()
            
            # 前向傳播
            cl, reg = model(batch.x, source_idx, target_idx)
            
            # 計算損失
            # 計算分類損失
            classification_loss = F.cross_entropy(
                cl, 
                batch.edge_label.long()
            )
            
            # 根據 input_id 獲取對應的 scores
            mask_pos = batch.edge_label == 1
            # 計算正樣本的 scores
            mask_reg = reg[mask_pos]
            # 計算回歸損失 (預測互動分數)
            regression_loss = F.mse_loss(
                mask_reg, 
                data.scores[batch.input_id]
            )
            
            # 組合兩種損失
            loss =  classification_loss +  0.01 * regression_loss

            # 反向傳播
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            total_iter += 1
    avg_loss = epoch_loss / total_iter
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')
    
    # Early stopping 檢查
    if avg_loss < best_loss - min_delta:
        best_loss = avg_loss
        patience_counter = 0
        # 保存最佳模型
        torch.save(model.state_dict(), 'node2vec_model_v4_180.pth')
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f'Early stopping triggered after {epoch + 1} epochs')
        break

/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and wil

Epoch 0, Average Loss: 5.7736


/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and wil

Epoch 1, Average Loss: 5.6186


/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and wil

Epoch 2, Average Loss: 5.6023


/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/home/bl515-ml/Documents/shaio_jie/sma/Kpop_challenge_analyze/.venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and wil

KeyboardInterrupt: 

In [22]:
# 加載最佳模型
model.load_state_dict(torch.load('node2vec_model_v4_180.pth'))

<All keys matched successfully>

In [23]:
# 只使用歷史數據
test_itter = collaboration_df[collaboration_df['timestamp'] >= datetime(2025, 1, 1).timestamp()]

all_recommendations = []
traget_idxs = torch.tensor([i for i in range(node_encoder.classes_.shape[0])], dtype=torch.long).t().to('cuda')

for _, itter_row in tqdm(test_itter.iterrows()):
    start_time = itter_row['timestamp'] - WINDOW_SIZE * 24 * 3600
    # 獲取當前窗口內的數據
    window_data = collaboration_df[
        (collaboration_df['timestamp'] >= start_time) & 
        (collaboration_df['timestamp'] < itter_row['timestamp'])
    ]
    
    
    if len(window_data) > 0:
        G = nx.DiGraph()
        
        # 建立節點
        for i in range(len(node_encoder.classes_)):
            G.add_node(i)
        
        # 计算边的权重
        idol_values = defaultdict(list)
        for _,row in window_data.iterrows():
            artist1_idx = node_encoder.transform([row['source']])[0]
            artist2_idx = node_encoder.transform([row['target']])[0]
            effectiveness = row['views'] * 0.5 + row['likes'] * 0.3 + row['comments'] * 0.2
            
            combination = (artist1_idx, artist2_idx)
            
            idol_values[combination].append(effectiveness)
        
        # 计算平均效益
        idol_avg = {name: sum(vals) / len(vals) for name, vals in idol_values.items()}
    
        # 更新图的权重
        for name, effectiveness in idol_avg.items():
            source_name, target_name = name
        
            distance = float('inf')
            if effectiveness and effectiveness > 0:
                distance = 1 / effectiveness
        
            G.add_edge(source_name, target_name, weight=distance, effectiveness=effectiveness)
        
            
        vec_G = node2vec.Graph(G, is_directed=True, p=0.5, q=1)
        vec_G.preprocess_transition_probs()
        walks = vec_G.simulate_walks(num_walks=1, walk_length=80)
        walks = [list(map(str, walk)) for walk in walks]
        embedding_model = Word2Vec(walks, vector_size=512, window=20, min_count=0, sg=1, workers=8, epochs=5, seed=42)
        
        # 轉換為張量
        node_features = torch.tensor(embedding_model.wv.vectors, dtype=torch.float).to('cuda')
        
        source = itter_row['source']
        
        source_idx = node_encoder.transform([source])[0]
        
        source_idxs = torch.tensor([source_idx for _ in range(traget_idxs.shape[0])], dtype=torch.long).t().to('cuda')
        
        with torch.no_grad():
            cl, reg = model(node_features, source_idxs, traget_idxs)
            
            softmax_cl = F.softmax(cl, dim=1)
            
            pos_probs = softmax_cl[:,1]

            topk_pos_vals, topk_pos_idx = torch.topk(pos_probs, k=10)
            
            topk_perf = reg[topk_pos_idx]
            sorted_perf, order    = torch.sort(topk_perf, descending=True)
            final_idx = topk_pos_idx[order]
            
            predictions = node_encoder.inverse_transform(final_idx.cpu().numpy())
            predictions_reg = reg[final_idx]
    
            all_recommendations.append({
                'source': source,
                'label': itter_row['target'],
                'score': itter_row['views'] * 0.5 + itter_row['likes'] * 0.3 + itter_row['comments'] * 0.2,
                'recommendations': predictions,
                'reg': predictions_reg.to('cpu').numpy()
            })
        
        

519it [05:51,  1.48it/s]


In [9]:
test_itter.iloc[0]['source']


'BOYNEXTDOOR_WOONHAK'

In [24]:
# 計算評估指標
true_positives = 0
total_recommendations = 0
reciprocal_ranks = []
hits_at_k = 0
mses = []
    
for recommendations in all_recommendations:
    rank = 0
    selected_reg = 0
    for i, n in enumerate(recommendations['recommendations']):
        if n == recommendations['label']:
            rank = i + 1
            selected_reg = recommendations['reg'][i]
            break;
    
    if rank > 0:
        reciprocal_ranks.append(1.0 / rank)
        mses.append((selected_reg - recommendations['score']) ** 2)
        if rank <= 10:
            hits_at_k += 1
    else:
        reciprocal_ranks.append(0.0)
    
    total_recommendations += len(recommendations['recommendations'])
    
        
mrr = np.mean(reciprocal_ranks) if reciprocal_ranks else 0
hit_at_k = hits_at_k / len(all_recommendations) if len(all_recommendations) > 0 else 0


mrr_percentage = mrr * 100
hit_at_k_percentage = hit_at_k * 100
print(f"MRR: {mrr_percentage:.4f} %, Hit@10: {hit_at_k_percentage:.4f} %, MSE: {np.mean(mses):.4f}")

MRR: 12.4286 %, Hit@10: 41.0405 %, MSE: 413.8528


In [12]:
last_df = collaboration_df[collaboration_df['timestamp'] >= collaboration_df['timestamp'].max() - WINDOW_SIZE * 24 * 3600]

result = []
G = nx.DiGraph()
        
 # 建立節點
for i in range(len(node_encoder.classes_)):
    G.add_node(i)
        
# 计算边的权重
idol_values = defaultdict(list)
for _,row in window_data.iterrows():
    artist1_idx = node_encoder.transform([row['source']])[0]
    artist2_idx = node_encoder.transform([row['target']])[0]
    effectiveness = row['views'] * 0.5 + row['likes'] * 0.3 + row['comments'] * 0.2
            
    combination = (artist1_idx, artist2_idx)
            
    idol_values[combination].append(effectiveness)
        
    # 计算平均效益
    idol_avg = {name: sum(vals) / len(vals) for name, vals in idol_values.items()}
    
# 更新图的权重
for name, effectiveness in idol_avg.items():
    source_name, target_name = name
        
    distance = float('inf')
    if effectiveness and effectiveness > 0:
        distance = 1 / effectiveness
        
    G.add_edge(source_name, target_name, weight=distance, effectiveness=effectiveness)
        
vec_G = node2vec.Graph(G, is_directed=True, p=0.5, q=1)
vec_G.preprocess_transition_probs()
walks = vec_G.simulate_walks(num_walks=1, walk_length=80)
walks = [list(map(str, walk)) for walk in walks]
embedding_model = Word2Vec(walks, vector_size=512, window=20, min_count=0, sg=1, workers=8, epochs=5, seed=42)
        
# 轉換為張量
node_features = torch.tensor(embedding_model.wv.vectors, dtype=torch.float).to('cuda')
    
source = itter_row['source']
predictions = []
with torch.no_grad():
    for i in node_encoder.classes_:
        source_idx = node_encoder.transform([i])
        for j in node_encoder.classes_:
            if j == i:
                continue
            target_idx = node_encoder.transform([j])
            cl, reg = model(data.x, data.edge_index, data.edge_attr, source_idx, target_idx)
            
            softmax_cl = F.softmax(cl, dim=1)
            
            is_positive = softmax_cl[:,1].item() > 0.5
            
            if is_positive:
                predictions.append((j, softmax_cl[:,1].item(), reg.item(), is_positive))

        predictions.sort(key=lambda x: x[1], reverse=True)
    
        top_k_recommendations = predictions[:10]
    
        top_k_recommendations = top_k_recommendations.sort(key=lambda x: x[2], reverse=True)
    
        result.append({
            'source': i,
            'recommendations': predictions
        })
    
# 創建預測結果 DataFrame
    predictions_df = pd.DataFrame(result)
    
    # 保存為 CSV 文件
    output_dir = 'predictions'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_file = os.path.join(output_dir, 'predictions_node2vec.csv')
    predictions_df.to_csv(output_file, index=False)
    print(f"\n預測結果已保存至: {output_file}")
    

TypeError: DownStreamModel.forward() takes 4 positional arguments but 6 were given